In [1]:
#libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.19.0'

In [ ]:
# Create an ImageDataGenerator object for training with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,              # Normalize pixel values from [0,255] to [0,1]
    shear_range=0.2,             # Randomly apply shearing transformations (tilting)
    zoom_range=0.3,              # Randomly zoom in on images up to 30%
    rotation_range=20,          # Randomly rotate images within a range of ±20 degrees
    width_shift_range=0.2,      # Shift images horizontally by up to 20% of the width
    height_shift_range=0.2,     # Shift images vertically by up to 20% of the height
    horizontal_flip=True,       # Randomly flip images horizontally
    brightness_range=[0.8,1.2]  # Randomly adjust brightness between 80% and 120%
)

# Load and preprocess images from the 'training' directory
training_set = train_datagen.flow_from_directory(
    'training',                  # Path to the training image folder (should have subfolders per class)
    target_size=(128, 128),      # Resize all images to 128x128 (match model input size)
    batch_size=32,               # Load images in batches of 32
    class_mode='categorical'     # Use one-hot encoded labels for multi-class classification
)


Found 1927 images belonging to 3 classes.


In [ ]:
# Create an ImageDataGenerator object for the test set (no augmentation, just rescaling)
test_datagen = ImageDataGenerator(
    rescale=1./255  # Normalize pixel values from [0,255] to [0,1]
)

# Load and preprocess images from the 'testing' directory
test_set = test_datagen.flow_from_directory(
    'testing',                  # Path to the test image folder (should have subfolders per class)
    target_size=(128, 128),     # Resize all images to 128x128 to match model input
    batch_size=32,              # Load images in batches of 32
    class_mode='categorical'    # Use one-hot encoded labels for multi-class classification
)


Found 225 images belonging to 3 classes.


In [ ]:
cnn = tf.keras.models.Sequential() #This line creates a Sequential model in Keras, which is a linear stack of layers.

In [ ]:
# Add the first convolutional layer to the CNN
cnn.add(tf.keras.layers.Conv2D(
    filters=32,                   # Number of output filters (i.e., 32 feature detectors)
    kernel_size=(3, 3),           # Size of the convolution window (3x3)
    activation='relu',           # ReLU activation function introduces non-linearity
    input_shape=[128, 128, 3]    # Shape of input images: 128x128 pixels, 3 color channels (RGB)
))
# Add a max pooling layer to reduce spatial dimensions and computation
cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=(2, 2)              # Size of the pooling window (2x2)
))


C:\Users\saza\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
cnn.add(tf.keras.layers.Conv2D(
    filters=64,                  # Number of filters (increased to 64 for deeper feature detection)
    kernel_size=(3, 3),          # Size of each filter (3x3 window)
    activation='relu'            # ReLU activation introduces non-linearity
))
cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=(2, 2)             # Downsamples the feature maps again by a factor of 2
))


In [ ]:
# Add a third convolutional layer to learn even more complex features
cnn.add(tf.keras.layers.Conv2D(
    128,                    # Number of filters (feature detectors) = 128
    (3, 3),                 # Size of each filter is 3x3
    activation='relu'       # ReLU activation to add non-linearity
))
# Add a third max pooling layer to reduce spatial dimensions further
cnn.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2)        # Pooling window size is 2x2
))


In [ ]:
# Flatten the 3D feature maps into a 1D feature vector
cnn.add(tf.keras.layers.Flatten())


In [ ]:
# Add a fully connected (dense) layer with 128 neurons and ReLU activation
cnn.add(tf.keras.layers.Dense(
    units=128,            # Number of neurons in this dense layer
    activation='relu'     # ReLU activation function to introduce non-linearity
))


In [ ]:
# Add a dropout layer to reduce overfitting
cnn.add(tf.keras.layers.Dropout(0.5))  # 50% of the neurons will be randomly turned off during training


In [ ]:
# Add the output layer with 3 neurons (one for each class) and softmax activation
cnn.add(tf.keras.layers.Dense(
    units=3,               # Number of output classes (Early blight, Late blight, Healthy)
    activation='softmax'   # Softmax converts outputs to probabilities that sum to 1
))


In [ ]:
# Compile the CNN model with optimizer, loss function, and evaluation metric
cnn.compile(
    optimizer='adam',                      # Adam optimizer adjusts learning rate during training
    loss='categorical_crossentropy',       # Suitable loss function for multi-class classification with one-hot encoded labels
    metrics=['accuracy']                   # Track accuracy during training and validation
)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define an EarlyStopping callback to prevent overfitting
early_stop = EarlyStopping(
    monitor='val_loss',           # Monitors the validation loss during training
    patience=5,                   # If val_loss doesn't improve for 5 consecutive epochs, stop training
    restore_best_weights=True     # After stopping, restore the model weights from the epoch with the best val_loss
)


In [ ]:
# Train the CNN model on the training data with validation and early stopping
cnn.fit(
    training_set,             # The training dataset with augmentation
    validation_data=test_set, # The validation dataset (used to monitor generalization)
    epochs=25,                # Train for up to 25 epochs (EarlyStopping may stop it earlier)
    callbacks=[early_stop]    # Apply early stopping to prevent overfitting
)


C:\Users\saza\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 20s 294ms/step - accuracy: 0.4715 - loss: 0.9458 - val_accuracy: 0.7067 - val_loss: 0.8246
Epoch 2/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 17s 275ms/step - accuracy: 0.7061 - loss: 0.6855 - val_accuracy: 0.6533 - val_loss: 0.9234
Epoch 3/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 17s 275ms/step - accuracy: 0.8371 - loss: 0.4182 - val_accuracy: 0.8178 - val_loss: 0.4737
Epoch 4/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 17s 280ms/step - accuracy: 0.8615 - loss: 0.3668 - val_accuracy: 0.7378 - val_loss: 0.8756
Epoch 5/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 17s 283ms/step - accuracy: 0.8977 - loss: 0.2626 - val_accuracy: 0.8444 - val_loss: 0.3958
Epoch 6/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 18s 291ms/step - accuracy: 0.9128 - loss: 0.2333 - val_accuracy: 0.8089 - val_loss: 0.4887
Epoch 7/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 17s 282ms/step - accuracy: 0.9003 - loss: 0.2713 - val_accuracy: 0.9067 - val_loss: 0.2289
Epoch 8/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 17s 280ms/step - accuracy: 0.9315 - loss: 0.1835 - val_accu

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Path to the image you want to predict
path = 'testing/Potato_Early_blight/0a8a68ee-f587-4dea-beec-79d02e7d3fa4___RS_Early.B 8461.JPG'

# Load the image and resize it to match the model's input size
test_image = image.load_img(path, target_size=(128,128))

# Convert the image to a NumPy array
test_image = image.img_to_array(test_image)

# Normalize the pixel values to range [0, 1] — same as during training
test_image = test_image / 255.0

# Expand dimensions to simulate a batch of 1 image (required by the model)
test_image = np.expand_dims(test_image, axis=0)

# Make the prediction
result = cnn.predict(test_image)

# Print class index mapping (e.g., {'Potato___Early_blight': 0, ...})
print("Class indices:", training_set.class_indices)

# Print the raw prediction vector (probabilities for each class)
print("Raw prediction:", result)

# Print the final predicted class index (e.g., 0, 1, or 2)
print("Predicted class:", np.argmax(result))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step


Class indices: {'Potato___Early_blight': 0, 'Potato___Late_blight': 1, 'Potato___healthy': 2}
Raw prediction: [[9.9486536e-01 5.1347064e-03 1.7583311e-12]]
Predicted class: 0


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

path = 'testing/Potato___Late_blight/0acdc2b2-0dde-4073-8542-6fca275ab974___RS_LB 4857.JPG'

# Load and preprocess the image
test_image = image.load_img(path, target_size=(128,128))  # Resize to match training size
test_image = image.img_to_array(test_image)                # Convert to array
test_image = test_image / 255.0                            # Normalize pixel values
test_image = np.expand_dims(test_image, axis=0)            # Add batch dimension

# Get prediction
result = cnn.predict(test_image)

# Output results
print("Class indices:", training_set.class_indices)        # Class mapping
print("Raw prediction:", result)                           # Softmax probabilities
print("Predicted class:", np.argmax(result))               # Final predicted class index


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Class indices: {'Potato___Early_blight': 0, 'Potato___Late_blight': 1, 'Potato___healthy': 2}
Raw prediction: [[0.0104459  0.9824254  0.00712869]]
Predicted class: 1


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Define the path to the test image you want to classify
path = 'testing/Potato_healthy/0be9d721-82f5-42c3-b535-7494afe01dbe___RS_HL 1814.JPG'

# Load the image from the path and resize it to 128x128 pixels (same as model input)
test_image = image.load_img(path, target_size=(128,128))

# Convert the loaded image to a NumPy array (shape: 128x128x3)
test_image = image.img_to_array(test_image)

# Normalize pixel values from [0,255] to [0,1] (same preprocessing as training)
test_image = test_image / 255.0

# Add an extra dimension to represent batch size = 1 (model expects shape: [batch, height, width, channels])
test_image = np.expand_dims(test_image, axis=0)

# Make prediction using the trained model
result = cnn.predict(test_image)

# Print the mapping of class labels to their corresponding indices
print("Class indices:", training_set.class_indices)

# Print the raw output of the softmax layer (probabilities for each class)
print("Raw prediction:", result)

# Print the index of the class with the highest predicted probability
print("Predicted class:", np.argmax(result))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Class indices: {'Potato___Early_blight': 0, 'Potato___Late_blight': 1, 'Potato___healthy': 2}
Raw prediction: [[3.5220297e-04 2.2415960e-02 9.7723186e-01]]
Predicted class: 2


In [ ]:
# Save the trained CNN model to a file named 'potato_leaf_classifier.h5'
cnn.save('potato_leaf_classifier.h5')


In [ ]:
from tensorflow.keras.models import load_model

# Load the previously saved CNN model from the .h5 file
model = load_model('potato_leaf_classifier.h5')


In [30]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Define the path to the test image
img_path = 'testing/Potato___Late_blight/0acdc2b2-0dde-4073-8542-6fca275ab974___RS_LB 4857.JPG'

# Load the image and resize it to the input size expected by the model (128x128 pixels)
img = image.load_img(img_path, target_size=(128, 128))

# Convert the image to a NumPy array (shape: 128x128x3)
img_array = image.img_to_array(img)

# Normalize pixel values to the [0, 1] range (same preprocessing as during training)
img_array = img_array / 255.0

# Expand dimensions to create a batch of 1 (model expects shape: [batch, height, width, channels])
img_array = np.expand_dims(img_array, axis=0)

# Use the loaded model to predict the class probabilities for the input image
result = model.predict(img_array)

# Get the index of the class with the highest probability
predicted_class_index = np.argmax(result)

# Define the list of class names in the same order as during training
class_names = ['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

# Print the predicted class name and the full probability distribution
print(f"Predicted class: {class_names[predicted_class_index]} ({result})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Predicted class: Potato___Late_blight ([[0.0104459  0.9824254  0.00712869]])
